# SageMaker PyTorch binary segmentation intro

Automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many jobs that test a range of hyperparameters on your dataset. You choose the tunable hyperparameters, a range of values for each, and an objective metric. You choose the objective metric from the metrics that the algorithm computes. Automatic model tuning searches the hyperparameters chosen to find the combination of values that result in the model that optimizes the objective metric.

## Introduction

This notebook demonstrates the use of the [PyTorch Segmentation models with pretrained backbones](https://github.com/qubvel/segmentation_models.pytorch) - Python library with Neural Networks for Image Segmentation based on [PyTorch](https://pytorch.org/).

The main features of this library are:

- High level API (just two lines to create a neural network)
- 9 models architectures for binary and multi class segmentation (including legendary Unet)
- 124 available encoders (and 500+ encoders from timm)
- All encoders have pre-trained weights for faster and better convergence
- Popular metrics and losses for training routines

This notebook shows how to use `segmentation-models-pytorch` for **binary** semantic segmentation. We will use the [The Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) (this is an adopted example from Albumentations package [docs](https://albumentations.ai/docs/examples/pytorch_semantic_segmentation/), which is strongly recommended to read, especially if you never used this package for augmentations before).

In [ ]:
!pip install sagemaker -U

In [ ]:
from sagemaker.pytorch import PyTorch
from datetime import datetime
import sagemaker

sess = sagemaker.Session()

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

print(f"Using SageMaker version {sagemaker.__version__}")
print(f"Using boto3 version {boto3.__version__}")

In [ ]:
training_dataset_s3_path = "s3://aws-ml-blog/artifacts/amazon-sagemaker-binary-segmentation-intro/oxford-pet-dataset"

In [ ]:
!aws s3 ls {training_dataset_s3_path}/

## Launching a training job with the Python SDK

In [ ]:
metric_definitions = [
    {'Name': 'test_dataset_iou', 'Regex': 'test_dataset_iou: ([0-9.]+).*$'},
    {'Name': 'test_per_image_iou', 'Regex': 'test_per_image_iou: ([0-9.]+).*$'},
]

In [ ]:
estimator = PyTorch(entry_point='train.py',
                        source_dir='./code',
                        role=role,
                        framework_version='1.10',
                        py_version='py38',
                        instance_count=1,
                        instance_type='ml.g5.2xlarge',
                        # keep_alive_period_in_seconds=3600,
                        metric_definitions=metric_definitions,
                        hyperparameters={
                            'epochs': 1,
                            'arch': "DeepLabV3Plus" ## Unet | FPN | DeepLabV3 | DeepLabV3Plus | Unet \ UnetPlusPlus
                        })
estimator.fit({"training": training_dataset_s3_path}, logs=True)

## Launching a tuning job with the Python SDK

In [ ]:
hpo_estimator = PyTorch(entry_point='train.py',
                        source_dir='./code',
                        role=role,
                        framework_version='1.10',
                        py_version='py38',
                        instance_count=1,
                        instance_type='ml.g5.2xlarge',
                        metric_definitions=metric_definitions
                        )

In [ ]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

hyperparameter_ranges = {
    "epochs": IntegerParameter(5, 20),
    "lr": ContinuousParameter(1e-4, 1e-3),
    "arch": CategoricalParameter(["FPN", 
                                  "DeepLabV3", 
                                  "DeepLabV3Plus", 
                                  "Unet", 
                                  "UnetPlusPlus", 
                                  "Linknet", 
                                  "PSPNet", 
                                  "PAN"])
}

In [ ]:
objective_metric_name = "test_dataset_iou"
objective_type = "Maximize"
hpo_metric_definitions = [
    {'Name': 'test_dataset_iou', 'Regex': 'test_dataset_iou: ([0-9.]+).*$'},
]

In [ ]:
tuner = HyperparameterTuner(
    hpo_estimator,
    objective_metric_name,
    hyperparameter_ranges,
    hpo_metric_definitions,
    max_jobs=24,
    max_parallel_jobs=4,
    objective_type=objective_type,
)

In [ ]:
tuner.fit({"training": training_dataset_s3_path})

## Get tuner results in a dataframe

In [ ]:
import pandas as pd

full_df = tuner.analytics().dataframe()
full_df.head()

In [ ]:
if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=False)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df